## Solving h-index problems

In [4]:
import numpy as np
import pandas as pd
import xlrd

In [5]:
# Loading Data for calculating hirsh-index

def hirsh(path):
    print("\n=== Индекс Хирша ===\n")
    print("Дано: \n")
    data = pd.read_excel(path,sheet_name=0, dtype=int)
    print(data)
    print("-----\nРешение:")
    h_index(data)
    
# Sorting table for h-index

def sorted_h_table(data):
    data["Цитирование"] = data.sort_values("Цитирование", ascending=False)["Цитирование"].values
    print(f'\nSorted Table:\n{data}')
    return data

# Calculating h-index from sorted table

def h_index(data):
    sorted_h_table(data)
    for index, row in data.iterrows():
        if row[1] < row[0]:
            print(f'\nh-index = {data.iloc[index-1,1]}')
            break

In [16]:
path = input(
    "Введите путь к файлу, либо название файла с расширением (если он в той же папке, что и скрипт): ")
try:
    hirsh(path)
except FileNotFoundError:
    print(f"ОШИБКА: Не найден Excel-файл: '{path}'. Вероятно, путь был указан некорректно. Проверьте, указано ли расширение файла (после названия файла в конце должно стоять .xlsx)")
except OSError:
    print(f"ОШИБКА: Путь не читаем: '{path}'. Вероятно это связано с тем, что вы ввели его через кавычки. Если это так, нужно кавычки убрать.")

Введите путь к файлу, либо название файла с расширением (если он в той же папке, что и скрипт): C:\Users\DLect\OneDrive\Рабочий стол\HSE\Research Seminar\research_seminar_problems.xlsx

=== Индекс Хирша ===

Дано: 

    Публикация  Цитирование
0            1           15
1            2           18
2            3            6
3            4            7
4            5            9
5            6            8
6            7           25
7            8           30
8            9           11
9           10            8
10          11            2
11          12            0
12          13            4
13          14            4
-----
Решение:

Sorted Table:
    Публикация  Цитирование
0            1           30
1            2           25
2            3           18
3            4           15
4            5           11
5            6            9
6            7            8
7            8            8
8            9            7
9           10            6
10          11            

## Непосредственная оценка

In [17]:
def direct_assessment_loading_data(path):
    print("\n=== Непосредственная оценка ===\n")
    print("Дано: \n")
    data2 = pd.read_excel(path,sheet_name=1, index_col=0)
    print(data2)
    return data2

def direct_assessment(data2):
    print("-----\nРешение:")
    data2_sum = data2.sum().sum()
    print(round(data2.sum()/data2_sum,2))
    
data2 = direct_assessment_loading_data(path)
direct_assessment(data2)


=== Непосредственная оценка ===

Дано: 

            K1   K2   K3   K4   K5   K6 
Эксперт 1    10    8    7    9    2    4
Эксперт 2     5    6    8    7    4   10
Эксперт 3     4    3    7    5    7    8
-----
Решение:
K1     0.17
K2     0.15
K3     0.19
K4     0.18
K5     0.11
K6     0.19
dtype: float64


## Парная оценка

In [18]:
def calculate_X(expert_matrix, n_experts):
    
    """
    Calculate Expected Value matrix from expert matrix
    
    """
    
    m = n_experts
    mi=1
    mj=0

    bool_matrix_mi = expert_matrix==mi
    bool_matrix_mi = bool_matrix_mi.astype(int)
    mi_matrix = sum(bool_matrix_mi)

    bool_matrix_mj = expert_matrix==mj
    bool_matrix_mj = bool_matrix_mj.astype(int)
    mj_matrix = sum(bool_matrix_mj)

    X = 0.5+(mi_matrix-mj_matrix)/(2*m)
    print(f'Матрица математических ожиданий оценок (X): {X}\n')
    
    return X


def calculate_coefficients(X, epsilon): 
    
    """
    Calculate coefficients from Expected Value matrix.
    
    """
    
    K = np.transpose(np.ones(X.shape[1]))
    while True:
        Y = np.dot(X, K)
        Lambda = np.dot(np.ones(X.shape[1]), Y)
        k = (1/Lambda)*Y
        if max(abs(k-K)) > epsilon:
            K = k
            print(f'Iteration: {K}')
            continue
        else:
            print(f'Итоговые значения коэффициента: {np.round(k,4)}')
            break

In [20]:
print("\n=== Парная оценка ===\n")
print("Дано: \n")

confidence_interval = float(input("Введите доверительный интервал "))
epsilon = 1 - confidence_interval

n_experts = int(input("Введите кол-во экспертов "))
matrix_dictionary = {}
for expert in range(0, n_experts):
    matrix_dictionary["expert{0}".format(expert+1)] = pd.read_excel(
        path, sheet_name=2, index_col=0, nrows=2, skiprows=expert*4).values
    
expert_matrix = np.stack(tuple(matrix_dictionary.values()))
print(f'\nМатрица экспертных оценок:\n {expert_matrix}\n')

print("-----\nРешение:\n")

X = calculate_X(expert_matrix, n_experts)
calculate_coefficients(X, epsilon)


=== Парная оценка ===

Дано: 

Введите доверительный интервал 0.99
Введите кол-во экспертов 4

Матрица экспертных оценок:
 [[[0.5 0.5]
  [0.5 0.5]]

 [[0.5 0. ]
  [1.  0.5]]

 [[0.5 0. ]
  [1.  0.5]]

 [[0.5 1. ]
  [0.  0.5]]]

-----
Решение:

Матрица математических ожиданий оценок (X): [[0.5   0.375]
 [0.625 0.5  ]]

Iteration: [0.4375 0.5625]
Итоговые значения коэффициента: [0.4365 0.5635]


## Анализ иерархий

In [21]:
def solve_hierarchy(criteria_data, object_matrix, n_criteria, n_objects):
    common_sum = np.sum(np.power(np.prod(criteria_data,axis = n_criteria-1),1/n_criteria))
    LK_array = np.power(np.prod(criteria_data,axis = n_criteria-1),1/n_criteria)/common_sum
    print(f'\nЛокальный приоритет критерия: {LK_array}')

    common_sum_objects = np.sum(np.power(np.prod(object_matrix,axis = n_objects-1),1/n_objects), axis=1)
    LK_objects_array = np.power(np.prod(object_matrix,axis = n_objects-1),1/n_objects)/common_sum_objects[:, np.newaxis]
    print(f'\nЛокальные приоритеты объектов относительно критериев:\n{LK_objects_array}')
    GK = np.dot(LK_array, LK_objects_array)
    print(f'\nГлобальные приоритеты: {GK}')


print("\n=== Анализ иерархий ===\n")
print("Дано: \n")

n_criteria = int(input("Введите кол-во критериев: "))
n_objects = int(input("Введите кол-во объектов: "))

matrix_dictionary = {}

criteria_data = pd.read_excel(
        path, sheet_name=3, index_col=0, nrows=2, usecols=list(range(0,n_criteria+1))).values
print(f'\nМатрица с оценкой критериев: \n{criteria_data}')

for criteria in range(0, n_criteria):
    matrix_dictionary["criteria{0}".format(criteria+1)] = pd.read_excel(
        "research_seminar_problems.xlsx", sheet_name=3, index_col=0, nrows=n_objects, skiprows=n_criteria+2+criteria*(n_objects+2)).values
    
object_matrix = np.stack(tuple(matrix_dictionary.values()))
print(f'\nОценка объектов по критериям:\n {object_matrix}\n')

print("-----\nРешение:")
    
solve_hierarchy(criteria_data, object_matrix, n_criteria, n_objects)    


=== Анализ иерархий ===

Дано: 

Введите кол-во критериев: 2
Введите кол-во объектов: 3

Матрица с оценкой критериев: 
[[1.  2.5]
 [0.4 1. ]]

Оценка объектов по критериям:
 [[[ 1.     8.     0.5  ]
  [ 0.125  1.     0.25 ]
  [ 2.     4.     1.   ]]

 [[ 1.     0.05   0.1  ]
  [20.     1.     4.   ]
  [10.     0.25   1.   ]]]

-----
Решение:

Локальный приоритет критерия: [0.71428571 0.28571429]

Локальные приоритеты объектов относительно критериев:
[[0.40677754 0.08071489 0.51250758]
 [0.02929508 0.73818973 0.23251519]]

Глобальные приоритеты: [0.2989254  0.26856484 0.43250975]


## Комплексная оценка

**Внимание!** Категориальные переменные, которые представлены в виде текста необходимо закодировать прямо в Excel, т.е. перевести их в числовые показатели! Также необходимо вручную убрать предметы, которые неэффективны (например, как это было с 3-м учебником).

In [22]:
def calculate_weighted_scores(matrix, length, V):
    mean_criteria = np.mean(matrix, axis=1)
    mean_criteria_matrix = np.transpose(
        np.vstack([mean_criteria]*length))
    abs_diff = np.sum(np.absolute(matrix - mean_criteria_matrix),
                      axis=1)/(length*mean_criteria)
    print(f"\nМатрица R: {abs_diff}")
    R_sum = np.sum(abs_diff)
    print(f"\nСумма R: {R_sum}")
    Z = abs_diff/R_sum
    print(f"\nМатрица Z: {Z}")
    W = (V+Z)/2
    print(f"\nМатрица W (обобщенные веса критериев): {W}")
    W_matrix = np.transpose(np.vstack([W]*length))
    answer = np.sum(W_matrix*matrix,axis=0)
    print(f'\nОтвет: {answer}')

In [27]:
print("\n=== Метод комплексной оценки ===\n")
print("Дано: \n")

df = pd.read_excel(path, sheet_name=4, index_col=0)
df1 = df.copy()

print(f'Список критериев: {df1.index.to_list()}')

minimize_crit = input(
    "Какие критерии минимизируем? Введите полное название критериев через запятую без пробела и кавычек: ").rsplit(",")

maximize_crit = input(
    "Какие критерии максимизируем? Введите полное название критериев через запятую без пробела и кавычек: ").rsplit(",")

V = input("Введите экспертные коэффициенты (веса) для каждого критерия по порядку (сверху-вниз) через запятую без пробела и кавычек: ").rsplit(",")
V = np.array(V)

try:
    V = V.astype(float)

    for crit in minimize_crit:
        df1.loc[crit] = min(df1.loc[crit])/df1.loc[crit]

    for crit in maximize_crit:
        df1.loc[crit] = df1.loc[crit] / max(df1.loc[crit])

    matrix = df1.values
    length = len(df1.columns)

    print("-----\nРешение:")

    calculate_weighted_scores(matrix, length, V)
except KeyError:
    print(f"ОШИБКА: Некорректно введены критерии. Проверьте, что название указанных критериев в точности совпадает с названиями в списке критериев. Также, проверьте введены ли критерии через запятую без пробела и кавычек.")
except ValueError:
    print(f"ОШИБКА: Некорректно введены экспертные коэффициенты.")


=== Метод комплексной оценки ===

Дано: 

Список критериев: ['Стоимость', 'Время реализации (год)', 'Эффективность']
Какие критерии минимизируем? Введите полное название критериев через запятую без пробела и кавычек: Стоимость,Время реализации (год)
Какие критерии максимизируем? Введите полное название критериев через запятую без пробела и кавычек: Эффективность
Введите экспертные коэффициенты (веса) для каждого критерия по порядку (сверху-вниз) через запятую без пробела и кавычек: 0.2,0.4,0.4
-----
Решение:

Матрица R: [0.42857143 0.         0.25      ]

Сумма R: 0.6785714285714286

Матрица Z: [0.63157895 0.         0.36842105]

Матрица W (обобщенные веса критериев): [0.41578947 0.2        0.38421053]

Ответ: [0.75052632 0.84631579]
